In [3]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import os
from scipy import misc

%matplotlib inline

In [26]:
def get_data(dirname):
    img_names = os.listdir(dirname)
    img_names = [f for f in img_names if f.endswith(".jpeg")]
    X = np.ndarray((len(img_names),512, 512, 3), dtype=np.float64)
    y = np.ndarray(len(img_names), dtype=np.float64)
    
    for num, img in enumerate(img_names):
        X[num] = misc.imread(dirname + "/" + img, mode='RGB')
        y[num] = float(img[0])
            
        num_testing = 5
        num_validation = 5
        num_training = len(img_names) - num_testing - num_validation
        # Subsample the data
        mask = range(num_training + num_testing, num_training + num_validation + num_testing)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training, num_training + num_testing)
        X_test = X[mask]
        y_test = y[mask]
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_data("images")

In [17]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [28]:
# clear old variables
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 512, 512, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def my_model(X,y,is_training):
    Wconv1 = tf.get_variable("Wconv1", shape=[5, 5, 3, 32])
    bconv1 = tf.get_variable("bconv1", shape=[32])
    Wconv2 = tf.get_variable("Wconv2", shape=[5, 5, 32, 32])
    bconv2 = tf.get_variable("bconv2", shape=[32])
    Wconv3 = tf.get_variable("Wconv3", shape=[5, 5, 32, 64])
    bconv3 = tf.get_variable("bconv3", shape=[64])
    
    W1 = tf.get_variable("W1", shape=[262144, 1024])
    b1 = tf.get_variable("b1", shape=[1024])
    W2 = tf.get_variable("W2", shape=[1024, 5])
    b2 = tf.get_variable("b2", shape=[5])

    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,1,1,1], padding='SAME') + bconv1
    p1 = tf.layers.max_pooling2d(a1, 2, 2)
    bn1 = tf.contrib.layers.batch_norm(p1, center=True, scale=True,
                                    is_training=is_training) # disbale reuse, not specify scope
    h1 = tf.nn.relu(bn1)
    
    a2 = tf.nn.conv2d(h1, Wconv2, strides=[1,1,1,1], padding='SAME') + bconv2
    p2 = tf.layers.max_pooling2d(a2, 2, 2)
    bn2 = tf.contrib.layers.batch_norm(p2, center=True, scale=True,
                                    is_training=is_training)
    h2 = tf.nn.relu(bn2)
    
    
    a3 = tf.nn.conv2d(h2, Wconv3, strides=[1,1,1,1], padding='SAME') + bconv3
    p3 = tf.layers.max_pooling2d(a3, 2, 2)
    bn3 = tf.contrib.layers.batch_norm(p3, center=True, scale=True,
                                    is_training=is_training)
    h3 = tf.nn.relu(bn3)
    
    # affine layer
    h5_flat = tf.reshape(h3,[-1,262144])
    h6_flat = tf.matmul(h5_flat,W1) + b1
    h7_flat = tf.nn.relu(h6_flat)
    y_out = tf.matmul(h7_flat,W2) + b2

    return y_out

# initialize the model
y_out = my_model(X,y,is_training)
mean_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_out, labels=tf.squeeze(y)))
optimizer = tf.train.RMSPropOptimizer(0.01)

# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)

In [ ]:
# trains for 10 epochs
# then prints the validation set accuracy
with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,y_out,mean_loss,X_train,y_train,10,64,100,train_step,True)
        print('Validation')
        run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

Training
